# `Tensor` vs `Scalar` performance comparison

Before moving on to the implementation details, let's quickly compare the performance of `Scalar` vs. `Tensor`. To do that, we'll use a simple one-layer neural network with thirty-two hidden neurons.

We'll store the number of hidden neurons inside a variable so you can play around with it if you execute this notebook locally. 

In [1]:
n_hidden_neurons = 32

First, here's the data. Feel free to skip the code, it's nothing we haven't seen before.

In [2]:
import numpy as np
from mlfz.nn.scalar import Scalar
from mlfz.nn.tensor import Tensor

n_samples = 1000

xs_scalar = [[Scalar.from_random(), Scalar.from_random()] for _ in range(n_samples)]
ys_scalar = [Scalar.from_random() for _ in range(n_samples)]

xs_tensor = Tensor(np.array([[x1.value, x2.value] for x1, x2 in xs_scalar]))
ys_tensor = Tensor(np.array([y.value for y in ys_scalar]).reshape(-1, 1))

Our `Scalar` network takes quite a while to set up. Here we go:

In [3]:
from mlfz import Model
from mlfz.nn.scalar import functional as f_scalar
from itertools import product


class ScalarNetwork(Model):
    def __init__(self):
        self.A = [[Scalar.from_random() for j in range(n_hidden_neurons)]
                  for i in range (2)]
        self.B = [Scalar.from_random() for i in range(n_hidden_neurons)]
    
    def forward(self, x):
        fs = [sum([self.A[i][j] * x[i] for i in range(2)]) for j in range(n_hidden_neurons)]
        fs_relu = [f_scalar.tanh(f) for f in fs]
        gs = sum([self.B[i] * fs_relu[i] for i in range(n_hidden_neurons)])
        return f_scalar.sigmoid(gs)

    def parameters(self):
        A_dict = {f"a{i}{j}": self.A[i][j] for i, j in product(range(2), range(n_hidden_neurons))}
        B_dict = {f"b{i}": self.B[i] for i in range(n_hidden_neurons)}
        return {**A_dict, **B_dict}

To accurately measure the time of a single gradient step, we encapsulate all the logic into a single function called `scalar_network_step`. 

In [4]:
from mlfz.nn.scalar.loss import binary_cross_entropy as bce_scalar


scalar_net = ScalarNetwork()


def scalar_network_step():
    preds = [scalar_net(x) for x in xs_scalar]
    l = bce_scalar(preds, ys_scalar)
    l.backward()
    scalar_net.gradient_update(0.01)
    

Let's go and %timeit!

In [5]:
%timeit scalar_network_step()

317 ms ± 6.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Now, about the `Tensor` network.

In [6]:
from mlfz.nn.tensor import functional as f_tensor


class TensorNetwork(Model):
    def __init__(self):
        self.A = Tensor.from_random(2, n_hidden_neurons)
        self.B = Tensor.from_random(n_hidden_neurons, 1)

    def forward(self, x):
        return f_tensor.sigmoid(f_tensor.tanh(x @ self.A) @ self.B)
    
    def parameters(self):
        return {"A": self.A, "B": self.B}

Look at that simplicity! Vectorization is worth it for that alone, but wait until we see how fast it is.

In [7]:
from mlfz.nn.tensor.loss import binary_cross_entropy as bce_tensor


tensor_net = TensorNetwork()


def tensor_network_step():
    preds = tensor_net(xs_tensor)
    l = bce_tensor(preds, ys_tensor)
    l.backward()
    tensor_net.gradient_update(0.01)

In [8]:
%timeit tensor_network_step()

3.46 ms ± 251 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The actual performance depends on the server this notebook is built on, but you should see a roughly 100x speedup, given by the magic of vectorization. If you are running this notebook locally, try changing the `n_hidden_neurons` variable in the first executable cell in this notebook. You'll be surprised: the execution time of the `Scalar` version will rapidly increase, but the `Tensor` version will roughly stay the same!

That's because the graph structure adds a heavy overhead to our computations. We'll profile the code in a later version of the notebook, but this is because the actual computations like addition, multiplication, etc, are only a small portion of the training!

## Coming soon

In the later chapters of this documentation, we'll detail how `Tensor` is implemented.